In [1]:
import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from pandas import DataFrame

nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cacki\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
input_text= """A window into the soul.For most people, this would be the eyes. The eyes cannot lie; they often tell more about a person's emotions than their words. What distinguishes a fake smile from a genuine one? The eyes. What shows sadness? The eyes. What gives away a liar? The eyes.But are the eyes the only window into the soul?Recently, I began painting with watercolors. With watercolors, there is no turning back: if one section is too dark, it is nearly impossible to lighten the area again. Every stroke must be done purposefully, every color mixed to its exact value.I laid my materials before me, preparing myself for the worst. I checked my list of supplies, making sure my setup was perfect.I wet my brush, dipped it into some yellow ochre, and dabbed off the excess paint. Too little water on my brush. I dipped my brush back into my trusty water jar; the colors swirled beautifully, forming an abstract art piece before my eyes. \u2014It's a shame that I couldn't appreciate it.I continued mixing colors to their exact value. More alizarin crimson. More water. More yellow ochre. Less water. More phthalo blue. The cycle continued. Eventually, I was satisfied. The colors looked good, there was enough contrast between facial features, and the watercolors stayed inside the lines.Craving feedback, I posted my art to Snapchat. I got a few messages such as 'wow' and 'pretty,' but one message stood out. 'You were anxious with this one, huh? Anyways, love the hair!'I was caught off guard. Was it a lucky guess? Did they know something I didn't? I immediately responded: 'Haha, how could you tell?' No response.What I didn't know at the time was that my response would come a few months later while babysitting. Since the girl I was babysitting loved art, I took out some Crayola watercolors and some watercolor paper for her to play with. After I went to the bathroom and came back, the watercolors were doused with water. 'You were impatient with this one, huh? Anyways, love the little dog you drew!'The little girl looked up at me, confused. 'How could you tell?' 'You used a lot of water for a brighter color, but you couldn't wait for it to slowly soak in.''Oh.'Now, I would be lying if I said I realized the connection between the two events immediately.Instead, I made the connection when I decided to sit down one day and objectively critique my art. The piece that I once loved now seemed like a nervous wreck: the paper was overworked, the brushstrokes were undecided, the facial features blended together, and each drop of water was bound inside the lines as if it was a prisoner in a cage.From then on, I started noticing pieces of personality in additional creations surrounding me: website designs, solutions to math problems, code written for class, and even the preparation of a meal.When I peer around at people's projects during Code Club, I notice the clear differences between their code. Some people break it up by commenting in every possible section. Others breeze through the project, not caring to comment or organize their code. I could also see clear differences in personalities when our club members began coding the Arduino for the first time. Some followed the tutorials to the letter, while others immediately started experimenting with different colored LEDs and ways of wiring the circuit.It became clear to me that, as humans, we leave pieces of our souls in everything we do, more than we intend to. If we entertain this thought, perhaps the key to better understanding others around us is simply noticing the subtler clues under our noses?Perhaps there are endless windows to the soul, and we simply need to peer through them. I shakily rose my hand. 'We should create workshops of our own,' I suggested.I got a few strange looks. 'It's a good idea, but it's too much work.' 'We just don't have enough free time to make it work.' 'Maybe we could, but I don't know how to make workshops.' My suggestion was shot down. I shuffled in my seat. 'I could make them.' A few people stared at me in disbelief. I glanced over at the club advisor, Mr. C, nervous to hear his response.'If you're willing to take on the work, we can try it.' Mr. C replied. And so I embarked on my quest. I researched different workshops on the internet, learning the information myself at first. Then, I transitioned into creating workshops of my own, making sure that the information was easy to understand for even a beginner. I was exhausted; my first workshop took 16 cumulative hours to create."""

## 종합분석 코드

In [3]:
def ext_character_sim_words(text):

    essay_input_corpus = str(text) #문장입력
    essay_input_corpus = essay_input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
    total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
    total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)

    skip_gram = 1
    workers = multiprocessing.cpu_count()
    bigram_transformer = Phrases(split_sentences)

    model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

    model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
    
    #모델 설계 완료

    #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
    character_list = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                      'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                    'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                    'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                    'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                    'twin','uncle','widow','widower','wife','ex-wife']
    
    ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
    #우선 토큰화한다.
    retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
    token_input_text = retokenize.tokenize(essay_input_corpus)
    #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
    #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
    filtered_chr_text = []
    for k in token_input_text:
        for j in character_list:
            if k == j:
                filtered_chr_text.append(j)
    
    #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
    
    filtered_chr_text_ = set(filtered_chr_text) #중복제거
    filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
    print (filtered_chr_text__) # 중복값 제거 확인
    
    for i in filtered_chr_text__:
        ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
    
    char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
    char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
        
    result_char_ratio = round(char_total_count/total_words * 100, 2)
    return result_char_ratio, total_sentences, total_words, char_total_count, char_count_, ext_sim_words_key

In [4]:
ext_character_sim_words(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력

df = ext_character_sim_words(input_text)
df_ = pd.DataFrame(df, index = ['charter ratio','total sentences', 'toral words', 'char_total_count', 'ext_chatacter_no', 'ext_similar_words'])
df_ # 이 겨래과에서는 92라는 숫자만 의미가 있지 >>> 총 문장에 캐릭터를 표현한 수가 총 몇개인지 추출한 결과

['i', 'them', 'they', 'you', 'her', 'me', 'it', 'my', 'their', 'myself']


C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).


['i', 'them', 'they', 'you', 'her', 'me', 'it', 'my', 'their', 'myself']


,0
charter ratio,9.87
total sentences,60
toral words,932
char_total_count,92
ext_chatacter_no,10
ext_similar_words,"[(learning, 0.9752081632614136), (preparing, 0..."


## Number of Characters
 of characters -> 합격 에세이 평균 내서 (3명?) 그것보다 큰지 작은지

In [5]:
def NumberofCharacters(text):

    essay_input_corpus = str(text) #문장입력
    essay_input_corpus = essay_input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
    total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
    total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)

    skip_gram = 1
    workers = multiprocessing.cpu_count()
    bigram_transformer = Phrases(split_sentences)

    model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

    model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
    
    #모델 설계 완료

    #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
    character_list = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                      'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                    'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                    'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                    'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                    'twin','uncle','widow','widower','wife','ex-wife']
    
    ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
    #우선 토큰화한다.
    retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
    token_input_text = retokenize.tokenize(essay_input_corpus)
    #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
    #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
    filtered_chr_text = []
    for k in token_input_text:
        for j in character_list:
            if k == j:
                filtered_chr_text.append(j)
    
    #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
    
    filtered_chr_text_ = set(filtered_chr_text) #중복제거
    filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
    print (filtered_chr_text__) # 중복값 제거 확인
    
    for i in filtered_chr_text__:
        ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
    
    char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
    char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
        
    result_char_ratio = round(char_total_count/total_words * 100, 2)
    return char_total_count

In [6]:
number_of_characters = NumberofCharacters(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
number_of_characters
print ('=============================================')
print ('Number of Characters :', number_of_characters)
print ('=============================================')

['i', 'them', 'they', 'you', 'her', 'me', 'it', 'my', 'their', 'myself']
Number of Characters : 92


C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).


## Emphasis on YOU 
'i'가 얼마나 들어갔는지, 평균대비

In [7]:
def EmphasisOnYou(text):

    essay_input_corpus = str(text) #문장입력
    essay_input_corpus = essay_input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
    total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
    total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)

    skip_gram = 1
    workers = multiprocessing.cpu_count()
    bigram_transformer = Phrases(split_sentences)

    model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

    model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
    
    #모델 설계 완료

    #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
    character_list = ['i', 'I', 'my', 'me', 'mine']
    
    ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
    #우선 토큰화한다.
    retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
    token_input_text = retokenize.tokenize(essay_input_corpus)
    #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
    #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
    filtered_chr_text = []
    for k in token_input_text:
        for j in character_list:
            if k == j:
                filtered_chr_text.append(j)
    
    #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
    
    filtered_chr_text_ = set(filtered_chr_text) #중복제거
    filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
    print (filtered_chr_text__) # 중복값 제거 확인
    
    for i in filtered_chr_text__:
        ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
    
    char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
    char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
        
    result_char_ratio = round(char_total_count/total_words * 100, 2)
    return char_total_count

In [8]:
EmphasisOnYou_ = EmphasisOnYou(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
EmphasisOnYou_
print ('=============================================')
print ('Emphasis on You :', EmphasisOnYou_)
print ('=============================================')

['me', 'i', 'my']
Emphasis on You : 60


C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:47: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).


## Emphasis on others
다른이들이 얼마나 들어갔는지, 평균대비

In [9]:
def EmphasisOnOthers(text):

    essay_input_corpus = str(text) #문장입력
    essay_input_corpus = essay_input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
    total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
    total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)

    skip_gram = 1
    workers = multiprocessing.cpu_count()
    bigram_transformer = Phrases(split_sentences)

    model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

    model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
    
    #모델 설계 완료

    #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
    character_list = ['you', 'your', 'they','them',
                      'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                    'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                    'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                    'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                    'twin','uncle','widow','widower','wife','ex-wife']
    
    ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
    #우선 토큰화한다.
    retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
    token_input_text = retokenize.tokenize(essay_input_corpus)
    #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
    #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
    filtered_chr_text = []
    for k in token_input_text:
        for j in character_list:
            if k == j:
                filtered_chr_text.append(j)
    
    #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
    
    filtered_chr_text_ = set(filtered_chr_text) #중복제거
    filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
    print (filtered_chr_text__) # 중복값 제거 확인
    
    for i in filtered_chr_text__:
        ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
    
    char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
    char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
        
    result_char_ratio = round(char_total_count/total_words * 100, 2)
    return char_total_count

In [10]:
EmphasisOnOthers_ = EmphasisOnOthers(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
EmphasisOnOthers_
print ('=============================================')
print ('Emphasis on Others :', EmphasisOnOthers_)
print ('=============================================')

['them', 'they', 'you', 'her', 'it', 'their', 'myself']
Emphasis on Others : 32


C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).


## Character Descriptiveness
이건 character 설명 문장에 showing이 얼마나 들어가 있는지… 평균대비

In [11]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F
import nltk
#cuda 메모리에 여유를 주기 위해서 잠시 딜레이를 시키자
import time

## Character 단어가 들어간 문장 추출

In [12]:
def findSentence(input_sentence):
    result = []

    data = str(input_sentence)
    #data = input_sentence.splitlines()
    
    findText = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                'twin','uncle','widow','widower','wife','ex-wife']

    sentences = data.split(".")
    
    for sentence in sentences:
        for item in findText:
            if item in sentence:
                result.append(sentence)

    return result

In [13]:
input_sent_included_character = findSentence(input_text) 
input_sent_chr = set(input_sent_included_character) #중복값을 제거해보자
input_sent_chr = '.'.join(input_sent_chr) #하나의 문자열로 합쳐야 원본 문장처럼 변환되고, 이것을 show/tell 분석코드에 넣게됨
input_sent_chr

" What distinguishes a fake smile from a genuine one? The eyes. If we entertain this thought, perhaps the key to better understanding others around us is simply noticing the subtler clues under our noses?Perhaps there are endless windows to the soul, and we simply need to peer through them. C replied. I got a few messages such as 'wow' and 'pretty,' but one message stood out. I dipped my brush back into my trusty water jar; the colors swirled beautifully, forming an abstract art piece before my eyes.Instead, I made the connection when I decided to sit down one day and objectively critique my art. What gives away a liar? The eyes. Too little water on my brush. Others breeze through the project, not caring to comment or organize their code.' My suggestion was shot down. I researched different workshops on the internet, learning the information myself at first. —It's a shame that I couldn't appreciate it.I continued mixing colors to their exact value. I could also see clear differences in

In [14]:
#입력된 전체 문장을 개별문장으로 분리하여 전처리 처리함
def sentence_to_df(input_sentence):

    input_text_df = nltk.tokenize.sent_tokenize(input_sentence)
    test = []

    for i in range(0,len(input_text_df)):
        new_label = np.random.randint(0,2)  # 개별문장(input_text_df) 수만큼 0 또는 1 난수 생성
        data = [new_label, input_text_df[i]]
        test.append(data)

    #print(test)
    dataf = pd.DataFrame(test, columns=['label', 'text'])
    #print(dataf)
    return dataf

In [15]:
class STDataset(Dataset):
    ''' Showing Telling Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = int(self.df.iloc[idx, 0])
        return text, label

In [16]:
###########입력받은 데이터 처리 실행하는 메소드 showtell_classfy() ###############
#result_all.html에서 입력받을 text를 contents에 넣고 전처리 후 데이터프레임에 넣어줌
def showtell_classfy(text):
    contents = str(text)
    preprossed_contents_df = sentence_to_df(contents)

    preprossed_contents_df.dropna(inplace=True)
    #전처리된 데이터를 확인(데이터프레임으로 가공됨)
    preprossed_contents_df__ = preprossed_contents_df.sample(frac=1, random_state=999)
    

    #파이토치에 입력하기 위해서 로딩...
    ST_test_dataset = STDataset(preprossed_contents_df__)
    test_loader = DataLoader(ST_test_dataset, batch_size=1, shuffle=True, num_workers=0)
    #로딩되는지 확인
    ST_test_dataset.__getitem__(1)

    #time.sleep(1)



    #check whether cuda is available
    #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
    device = torch.device("cpu")  
    #device = torch.device("cuda")
    #tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
    model = BertForSequenceClassification.from_pretrained('bert-large-cased')
    model.to(device)



    # for text, label in test_loader :
    #     print("text:",text)
    #     print("label:",label)


    #저장된 모델을 불러온다.
    #J:\Django\EssayFit_Django\essayfitaiproject\essayfitapp\model.pt
    #time.sleep(1)
    #model = torch.load("/Users/jongphilkim/Desktop/Django_WEB/essayfitaiproject/essayai/model.pt", map_location=torch.device('cpu'))
    model = torch.load("model.pt", map_location=torch.device('cpu'))
    print("model loadling~")
    model.eval()


    pred_loader = test_loader
    print("pred_loader:", pred_loader)
    total_loss = 0
    total_len = 0
    total_showing__ = 0
    total_telling__ = 0

    showing_conunter = [] #문장에 해당하는 SHOWING을 계산한다.
    
    print("check!")
    for text, label in pred_loader:
        print("text:",text)
        #print("label:",label)
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text] #text to tokenize
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list] #padding
        sample = torch.tensor(padded_list) #torch tensor로 변환
        sample, label = sample.to(device), label.to(device) #tokenized text에 label을 넣어서 Device(gpu/cpu)에 넣기 위해 준비
        labels = torch.tensor(label) #레이블을 텐서로 변환
        #time.sleep(1)
        outputs = model(sample,labels=labels) #모델을 통해서 샘플텍스트와 레이블 입력데이터를 출력 output에 넣음
        #시간 딜레이를 주자
        #time.sleep(1)
        _, logits = outputs #outputs를 로짓에 넣음 이것을 softmax에 넣으면 0~1 사이로 결과가 출력됨
        
        pred = torch.argmax(F.softmax(logits), dim=1) #드디어 예측한다. argmax는 리스트(계산된 값)에서 가장 큰 값을 추출하여 pred에 넣는다. 0 ~1 사이의 값이 나올거임
        print('pred :', pred)
        # correct = pred.eq(labels) 
        showing__ = pred.eq(1) # 예측한 결과가 1과 같으면 showing이다   >> TRUE   SHOWING을 추출하려면 이것만 카운드하면 된다. 
        telling__ = pred.eq(0) # 예측한 결과가 0과 같으면 telling이다   >> FALSE
        
        #print('showing : ', showing__)
        #print('telling : ', telling__)
        
        
        showing_conunter.append(text)        
        #pred_ = round(float(pred))
        showing_conunter.append(pred)



    return showing_conunter 

In [17]:
input_sent_chr # 입력값 다시 확인

" What distinguishes a fake smile from a genuine one? The eyes. If we entertain this thought, perhaps the key to better understanding others around us is simply noticing the subtler clues under our noses?Perhaps there are endless windows to the soul, and we simply need to peer through them. C replied. I got a few messages such as 'wow' and 'pretty,' but one message stood out. I dipped my brush back into my trusty water jar; the colors swirled beautifully, forming an abstract art piece before my eyes.Instead, I made the connection when I decided to sit down one day and objectively critique my art. What gives away a liar? The eyes. Too little water on my brush. Others breeze through the project, not caring to comment or organize their code.' My suggestion was shot down. I researched different workshops on the internet, learning the information myself at first. —It's a shame that I couldn't appreciate it.I continued mixing colors to their exact value. I could also see clear differences in

In [18]:
#input_text= """A window into the soul.For most people, this would be the eyes. The eyes cannot lie; they often tell more about a person's emotions than their words. What distinguishes a fake smile from a genuine one? The eyes. What shows sadness? The eyes. What gives away a liar? The eyes.But are the eyes the only window into the soul?Recently, I began painting with watercolors. With watercolors, there is no turning back: if one section is too dark, it is nearly impossible to lighten the area again. Every stroke must be done purposefully, every color mixed to its exact value.I laid my materials before me, preparing myself for the worst. I checked my list of supplies, making sure my setup was perfect.I wet my brush, dipped it into some yellow ochre, and dabbed off the excess paint. Too little water on my brush. I dipped my brush back into my trusty water jar; the colors swirled beautifully, forming an abstract art piece before my eyes. \u2014It's a shame that I couldn't appreciate it.I continued mixing colors to their exact value. More alizarin crimson. More water. More yellow ochre. Less water. More phthalo blue. The cycle continued. Eventually, I was satisfied. The colors looked good, there was enough contrast between facial features, and the watercolors stayed inside the lines.Craving feedback, I posted my art to Snapchat. I got a few messages such as 'wow' and 'pretty,' but one message stood out. 'You were anxious with this one, huh? Anyways, love the hair!'I was caught off guard. Was it a lucky guess? Did they know something I didn't? I immediately responded: 'Haha, how could you tell?' No response.What I didn't know at the time was that my response would come a few months later while babysitting. Since the girl I was babysitting loved art, I took out some Crayola watercolors and some watercolor paper for her to play with. After I went to the bathroom and came back, the watercolors were doused with water. 'You were impatient with this one, huh? Anyways, love the little dog you drew!'The little girl looked up at me, confused. 'How could you tell?' 'You used a lot of water for a brighter color, but you couldn't wait for it to slowly soak in.''Oh.'Now, I would be lying if I said I realized the connection between the two events immediately.Instead, I made the connection when I decided to sit down one day and objectively critique my art. The piece that I once loved now seemed like a nervous wreck: the paper was overworked, the brushstrokes were undecided, the facial features blended together, and each drop of water was bound inside the lines as if it was a prisoner in a cage.From then on, I started noticing pieces of personality in additional creations surrounding me: website designs, solutions to math problems, code written for class, and even the preparation of a meal.When I peer around at people's projects during Code Club, I notice the clear differences between their code. Some people break it up by commenting in every possible section. Others breeze through the project, not caring to comment or organize their code. I could also see clear differences in personalities when our club members began coding the Arduino for the first time. Some followed the tutorials to the letter, while others immediately started experimenting with different colored LEDs and ways of wiring the circuit.It became clear to me that, as humans, we leave pieces of our souls in everything we do, more than we intend to. If we entertain this thought, perhaps the key to better understanding others around us is simply noticing the subtler clues under our noses?Perhaps there are endless windows to the soul, and we simply need to peer through them. I shakily rose my hand. 'We should create workshops of our own,' I suggested.I got a few strange looks. 'It's a good idea, but it's too much work.' 'We just don't have enough free time to make it work.' 'Maybe we could, but I don't know how to make workshops.' My suggestion was shot down. I shuffled in my seat. 'I could make them.' A few people stared at me in disbelief. I glanced over at the club advisor, Mr. C, nervous to hear his response.'If you're willing to take on the work, we can try it.' Mr. C replied. And so I embarked on my quest. I researched different workshops on the internet, learning the information myself at first. Then, I transitioned into creating workshops of my own, making sure that the information was easy to understand for even a beginner. I was exhausted; my first workshop took 16 cumulative hours to create."""

In [20]:
#st_re = showtell_classfy(input_text) # 캐릭터거 포함된 문장(전처리 완료된) 입력
st_re = showtell_classfy(str(input_sent_chr)) # 캐릭터거 포함된 문장(전처리 완료된) 입력
st_re

model loadling~
pred_loader: <torch.utils.data.dataloader.DataLoader object at 0x000001ACEBA15EC8>
check!
text: ('With watercolors, there is no turning back: if one section is too dark, it is nearly impossible to lighten the area again.From then on, I started noticing pieces of personality in additional creations surrounding me: website designs, solutions to math problems, code written for class, and even the preparation of a meal.',)


C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


pred : tensor([1])
text: ("'We just don't have enough free time to make it work.",)
pred : tensor([0])
text: ('Some people break it up by commenting in every possible section.',)
pred : tensor([1])
text: ('I was exhausted; my first workshop took 16 cumulative hours to create.',)
pred : tensor([1])
text: ('I researched different workshops on the internet, learning the information myself at first.',)
pred : tensor([1])
text: ('More alizarin crimson.',)
pred : tensor([0])
text: ('No response.I laid my materials before me, preparing myself for the worst.',)
pred : tensor([1])
text: ("Did they know something I didn't?",)
pred : tensor([0])
text: ('Every stroke must be done purposefully, every color mixed to its exact value.',)
pred : tensor([1])
text: ('I dipped my brush back into my trusty water jar; the colors swirled beautifully, forming an abstract art piece before my eyes.Instead, I made the connection when I decided to sit down one day and objectively critique my art.',)
pred : tensor

[('With watercolors, there is no turning back: if one section is too dark, it is nearly impossible to lighten the area again.From then on, I started noticing pieces of personality in additional creations surrounding me: website designs, solutions to math problems, code written for class, and even the preparation of a meal.',),
 tensor([1]),
 ("'We just don't have enough free time to make it work.",),
 tensor([0]),
 ('Some people break it up by commenting in every possible section.',),
 tensor([1]),
 ('I was exhausted; my first workshop took 16 cumulative hours to create.',),
 tensor([1]),
 ('I researched different workshops on the internet, learning the information myself at first.',),
 tensor([1]),
 ('More alizarin crimson.',),
 tensor([0]),
 ('No response.I laid my materials before me, preparing myself for the worst.',),
 tensor([1]),
 ("Did they know something I didn't?",),
 tensor([0]),
 ('Every stroke must be done purposefully, every color mixed to its exact value.',),
 tensor([1]

In [21]:
type(st_re)

list

In [22]:
df = DataFrame(st_re)
df

,0
0,"With watercolors, there is no turning back: if..."
1,tensor(1)
2,'We just don't have enough free time to make i...
3,tensor(0)
4,Some people break it up by commenting in every...
...,...
103,tensor(1)
104,"After I went to the bathroom and came back, th..."
105,tensor(1)
106,"'If you're willing to take on the work, we can..."


In [23]:
df_ = df[0::2]
df_[:5] # 글만 추출

,0
0,"With watercolors, there is no turning back: if..."
2,'We just don't have enough free time to make i...
4,Some people break it up by commenting in every...
6,I was exhausted; my first workshop took 16 cum...
8,I researched different workshops on the intern...


In [24]:
df_label = df[1::2]
df_label # 레이블만 추출

,0
1,tensor(1)
3,tensor(0)
5,tensor(1)
7,tensor(1)
9,tensor(1)
11,tensor(0)
13,tensor(1)
15,tensor(0)
17,tensor(1)
19,tensor(1)


In [25]:
df_.reset_index(drop=True, inplace=True) #데이터를 합치기 위해서 초기화
df_label.reset_index(drop=True, inplace=True)

In [26]:
df_[:5]

,0
0,"With watercolors, there is no turning back: if..."
1,'We just don't have enough free time to make i...
2,Some people break it up by commenting in every...
3,I was exhausted; my first workshop took 16 cum...
4,I researched different workshops on the intern...


In [27]:
df_label[:5]

,0
0,tensor(1)
1,tensor(0)
2,tensor(1)
3,tensor(1)
4,tensor(1)


In [28]:
df_result = pd.concat([df_,df_label],axis=1)
df_result[:30]

,0,0
0,"With watercolors, there is no turning back: if...",tensor(1)
1,'We just don't have enough free time to make i...,tensor(0)
2,Some people break it up by commenting in every...,tensor(1)
3,I was exhausted; my first workshop took 16 cum...,tensor(1)
4,I researched different workshops on the intern...,tensor(1)
5,More alizarin crimson.,tensor(0)
6,"No response.I laid my materials before me, pre...",tensor(1)
7,Did they know something I didn't?,tensor(0)
8,"Every stroke must be done purposefully, every ...",tensor(1)
9,I dipped my brush back into my trusty water ja...,tensor(1)


In [29]:
df_result.columns = ['sentence','show/tell']
df_result

,sentence,show/tell
0,"With watercolors, there is no turning back: if...",tensor(1)
1,'We just don't have enough free time to make i...,tensor(0)
2,Some people break it up by commenting in every...,tensor(1)
3,I was exhausted; my first workshop took 16 cum...,tensor(1)
4,I researched different workshops on the intern...,tensor(1)
5,More alizarin crimson.,tensor(0)
6,"No response.I laid my materials before me, pre...",tensor(1)
7,Did they know something I didn't?,tensor(0)
8,"Every stroke must be done purposefully, every ...",tensor(1)
9,I dipped my brush back into my trusty water ja...,tensor(1)


In [30]:
df_fin = df_result['show/tell'].value_counts(normalize=True)
df_fin

tensor(1)    0.648148
tensor(0)    0.203704
tensor(0)    0.148148
Name: show/tell, dtype: float64

In [31]:
type(df_fin)

pandas.core.series.Series

In [32]:
list(df_fin)

[0.6481481481481481, 0.2037037037037037, 0.14814814814814814]

In [33]:
#이건 character 설명 문장에 showing이 얼마나 들어가 있는지… 평균대비

showing_sentence_with_char = max(round(df_fin*100))
showing_sentence_with_char

65.0

In [34]:
print ('Char_Showing Ratio in Sentence : ', showing_sentence_with_char)

Char_Showing Ratio in Sentence :  65.0
